In [2]:
from secret_codes import *

import psycopg2

In [3]:
# Create connection to test database
conn = psycopg2.connect(user="postgres",
                        password=PG_PASS,
                        host="localhost",
                        port=5432,
                        database="forest_test")

In [4]:
# Open cursor
cur = conn.cursor()

In [4]:
# Clean everything
cur.execute("DROP TABLE IF EXISTS forest_info, metrics, images, locations CASCADE;")

conn.commit()

In [6]:
# Create table 1 with forest info(T1)
cur.execute("""CREATE TABLE forest_info (forest_id SERIAL PRIMARY KEY,
                                        forest_name VARCHAR,
                                        is_active BOOLEAN,
                                        date_added TIMESTAMP WITH TIME ZONE default current_timestamp,
                                        date_planted DATE,
                                        geom geometry(POLYGON,4326)
                                        );"""
            )

conn.commit()

In [8]:
# Create auto-update trigger for date_added in T1
cur.execute("""CREATE OR REPLACE FUNCTION func_forest_date_added()
                RETURNS TRIGGER AS $$
                BEGIN
                    NEW.date_added = now();
                    RETURN NEW;
                END;
                $$ language 'plpgsql';"""
            )

# Create the trigger
cur.execute("""CREATE TRIGGER trig_forest_date_added
                BEFORE UPDATE ON forest_info
                FOR EACH ROW EXECUTE PROCEDURE func_forest_date_added();"""
            )
conn.commit()

In [ ]:
# Create table 2 (T2)
cur.execute("""CREATE TABLE metrics (metric_id INT PRIMARY KEY,
                                    forest_id INT,
                                    ndvi REAL,
                                    calc_date TIMESTAMP,
                                    CONSTRAINT fk_forest_id
                                        FOREIGN KEY(forest_id)
                                            REFERENCES forest_info(forest_id)
                                    );"""
            )

In [ ]:
# Create table 3
cur.execute("""CREATE TABLE images (images_id INT GENERATED ALWAYS AS IDENTITY,
                                    forest_id INT,
                                    r_band_url VARCHAR,
                                    g_band_url VARCHAR,
                                    b_band_url VARCHAR,
                                    scene_date TIMESTAMP WITH TIME ZONE,
                                    CONSTRAINT fk_forest_id
                                            FOREIGN KEY(forest_id)
                                                REFERENCES forest_info(forest_id)
                                    );"""
           )

In [5]:
# Create table 4
cur.execute("""CREATE TABLE locations (locations_id INT GENERATED ALWAYS AS IDENTITY,
                                        forest_id INT,
                                        loc_sys VARCHAR,
                                        loc_val VARCHAR,
                                        CONSTRAINT fk_forest_id
                                                FOREIGN KEY(forest_id)
                                                    REFERENCES forest_info(forest_id)
                                        );"""
           )

In [6]:
conn.commit()

In [ ]:
conn.close()

In [ ]:
cur.execute("ROLLBACK;")